# Deutsch's Problem

이번 글에서는 Quantum Computing이 Classical Computing에 비해 유리한 **Deutsch's Problem**에 대해 설명하겠다.  

1 또는 0 비트 정보를 입력하면 1 또는 0의 결과를 출력하는 함수가 있다고 하자.  
다음의 4가지 함수를 생각해 볼 수 있을 것이다.  


&nbsp;&nbsp;&nbsp;&nbsp;f(x) = 0         ...(1)  
&nbsp;&nbsp;&nbsp;&nbsp;f(x) = 1         ...(2)  
&nbsp;&nbsp;&nbsp;&nbsp;f(x) = x         ...(3)  
&nbsp;&nbsp;&nbsp;&nbsp;f(x) = $\bar{x}$ ...(4) 

(1), (2)는 입력값에 상관 없이 0 또는 1이 출력되는 **constant function**이고,  
(3), (4)는 입력값에 따라 입력값을 그대로, 또는 입력값의 NOT값을 출력하는 **balanced function**이다.

우리는 어떤 함수로 계산되는지 모르는 채로 입력값을 넣고 출력값을 받는다고 생각해보자.  
또, 이 함수의 연산 과정이 24시간 걸린다고 가정하자.  

우리는 이 함수가 **constant function**인지, **balanced function**인지 알아맞추고자 한다.

Classical Computing에서는 몇 시간이 필요할까?  
총 48 시간이 필요할 것이다. 예를 들어, 0을 입력하고 0이 나왔을 때, (1) 또는 (3) 함수 일 것이고, 또 1을 입력해서 0 또는 1이 나오는 지 확인해야 할 것이다.  
우리는 **constant function**인지 **balanced function**인지만 궁금했지만, 어쩔 수 없이 함수 자체를 찾은 것이다.

Quantum Computing으로는 24시간, 즉 한 번의 input으로 두 함수 유형을 구분할 수 있다.  
아래는 Deutsch's Problem을 재현한 코드다.

In [1]:
import numpy as np
import qiskit as q

secret_number = np.random.randint(1,5) # Random number from 1 to 4.

In [2]:
circ = q.QuantumCircuit(2,1)

circ.h(0) # Haddamard gate on 1st qubit.
circ.x(1) # Since qubit is initialized as |0>.
circ.h(1) # Haddamard gate on 2nd qubit.

if secret_number == 1:
    pass # Do nothing.
elif secret_number == 2:
    circ.x(1) # NOT gate on 2nd qubit.
elif secret_number == 3:
    circ.cx(0,1) # CNOT gate on 2nd qubit, based on 1st qubit.
else:
    circ.x(0) # NOT gate on 1st qubit.
    circ.cx(0,1) # CNOT gate on 2nd qubit, based on 1st qubit.
    circ.x(0) # NOT gate on 1st qubit.
    
circ.h(0)
circ.measure(0,0) # We only need to measure the 1st qubit.

In [3]:
backend = q.Aer.get_backend('qasm_simulator') # Run on 'qasm_simulator' backend.
result = q.execute(circ, backend, shots=1024).result() # 1024 shots.

counts = result.get_counts(circ)
print(counts)
for measured in counts: # Since we are running the process on simulator, perfect results are given.
    if not int(measured): print("1st qubit result: " + measured + ", Black Box had constant function!")
    else: print("1st qubit result: " + measured + ", Black Box had balanced function!")

{'1': 1024}
1st qubit result: 1, Black Box had balanced function!


In [4]:
# This is for final checking.
print("The secret number was: " + str(secret_number))
circ.draw()

The secret number was: 3


즉, 우리는 1st qubit을 measure함으로써, black box의 함수 유형을 확인할 수 있게 된다.

### 추가 설명.
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$Quantum Oracle은 함수 f에 대해 다음과 같이 정의된다.
$$U_{f}\ket{x}\ket{y} = \ket{x}\ket{y\oplus f(x)}$$
따라서,
1. f(x) = 0
$$U_{f}\ket{x}\ket{y} = \ket{x}\ket{y\oplus 0} = \ket{x}\ket{y}$$
$$U_{f} = I$$
2. f(x) = 1
$$U_{f}\ket{x}\ket{y} = \ket{x}\ket{y\oplus1} = I\otimes X\ket{x}\ket{y}$$
$$U_{f} = I\otimes X$$
3. f(x) = x
$$\ket{00} = \ket{00}$$
$$\ket{01} = \ket{01}$$
$$\ket{10} = \ket{11}$$
$$\ket{11} = \ket{10}$$
이므로, $U_{f} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}$
4. f(x) = $\bar{x}$
$$\ket{00} = \ket{01}$$
$$\ket{01} = \ket{00}$$
$$\ket{10} = \ket{10}$$
$$\ket{11} = \ket{10}$$
이므로, $U_{f} = \begin{pmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$

Deutschs algorithm은 다음과 같이 전개된다.
$$\ket{01}\;\underrightarrow{H_{2}}\;\frac{1}{2}(\ket{0} + \ket{1})(\ket{0}-\ket{1})$$
$$=\frac{1}{2}\ket{0}(\ket{0}-\ket{1}) + \frac{1}{2}\ket{1}(\ket{0}-\ket{1})$$
$$\underrightarrow{U_{f}}\;\frac{1}{2}(-1)^{f(0)}\ket{0}(\ket{0}-\ket{1}) + \frac{1}{2}(-1)^{f(1)}\ket{1}(\ket{0}-\ket{1})$$
$$=\frac{1}{2}((-1)^{f(0)}\ket{0} + (-1)^{f(1)}\ket{1})(\ket{0}-\ket{1})$$


만약 f가 constant function이면, 첫번째 qubit은 $$\pm\frac{1}{\sqrt{2}}(\ket{0}+\ket{1}) = \pm H(\ket{0})$$
f가 balanced function이면, 첫번째 qubit은 $$\pm\frac{1}{\sqrt{2}}(\ket{0}-\ket{1}) = \pm H(\ket{1})$$
일 것이다.